# Preguntas

El presente documento tiene como finalidad dar respuesta a las preguntas que surgen a partir del estudio anexo del paquete GO3, en su versión 0.3.0, de manera que quienes deseen utilizarlo en el futuro puedan comprender, en un grado razonable de profundidad, las características asociadas a esta librería. Adicionalmente, y de forma preventiva, se optó por definir algunos conceptos previos con el objetivo de facilitar la comprensión de la teoría asociada a la disciplina en la que se enmarca esta herramienta.

## Bioinformática y genes.
La **bioinformática** es una disciplina científica **interdisciplinaria** que combina biología, informática, estadística, matemáticas y ciencias de la computación para **adquirir, almacenar, gestionar, analizar e interpretar grandes conjuntos de datos biológicos**. Estos datos suelen corresponder a secuencias de ADN, ARN, proteínas y otros tipos de información molecular que requieren herramientas computacionales para ser procesados de manera eficiente y significativa [1] [2] [3].

En otras palabras, la bioinformática surge como respuesta a la enorme cantidad y complejidad de información generada por técnicas modernas como la secuenciación de genomas, y está diseñada para resolver problemas biológicos que serían imposibles de abordar sin el apoyo de métodos computacionales avanzados. A diferencia de lo que se conoce como **biología computacional**, que se enfoca con mayor frecuencia en la **modelización y simulación matemática de sistemas biológicos complejos**, la bioinformática está orientada principalmente al **análisis de datos biológicos**, al desarrollo de **algoritmos y _software_** que permitan extraer patrones y conocimiento a partir de esos datos [1] [2] [3].

Una parte central de la bioinformática es el estudio de las secuencias de genes y sus productos, lo cual está relacionado con el **dogma central de la biología molecular** (ADN → ARN → proteína), ya que muchas de las herramientas bioinformáticas se aplican precisamente a estos tipos de datos para entender funciones, variaciones y relaciones evolutivas.

<figure style="text-align:center;">
    <img src="Images/DogmaBM.png" width="60%">
    <figcaption>Figura 1: Dogma Central de la Biológia Molecular [8]</figcaption>
</figure>

El **ADN (ácido desoxirribonucleico)** es el material genético que contiene la información necesaria para el funcionamiento, desarrollo y reproducción de los seres vivos. Esta información está organizada en estructuras llamadas **genes**, que son segmentos de ADN que codifican instrucciones para **producir productos funcionales, como proteínas o ARN**. Cada gen está formado por una secuencia de nucleótidos que, al ser leídos en grupos de tres (llamados codones), determinan qué aminoácidos se ensamblan para formar una proteína. Sin embargo, la relación entre genes y proteínas no es estrictamente uno a uno. Un solo gen puede producir más de una proteína mediante procesos como el _splicing_ alternativo, y distintos genes pueden colaborar para la producción de la misma proteína o de componentes relacionados [4] [5] [6].

Un ejemplo común sobre lo anteriormente mencionado era la creencia de que el pelo rojo y piel clara poseen relación con la presencia de pecas en la piel (melanomas cutaneos), aunque cabe señalar que esta aseveración ya se confirmo en el año 2001 que esto no es una coincidencia absoluta [7]

<figure style="text-align:center;">
    <img src="Images/Ilustración_peliroja.jpg" width="20%">
    <figcaption>Figura 2: Ilustración_peliroja [9]</figcaption>
</figure>

## Semántica, DAG y Gene Ontology.
La **semántica** es una rama de la lingüística dedicada al **estudio del significado de las expresiones lingüísticas, tales como palabras, sintagmas y oraciones completas**. Desde un punto de vista formal, suele dividirse en dos grandes enfoques: la **semántica léxica**, que estudia el significado de las palabras de manera individual, y la **semántica composicional**, que analiza cómo el significado de expresiones más complejas se construye a partir del significado de sus componentes y de las reglas gramaticales que los relacionan [10] [11].

Dentro de este marco teórico, la semántica cumple un rol fundamental en la **representación y relación de conceptos**, especialmente cuando estos se organizan en estructuras formales como los grafos dirigidos acíclicos (_Directed Acyclic Graphs_, _DAG_). En este tipo de estructuras, los conceptos se conectan mediante relaciones jerárquicas que van desde **conceptos raíz, de carácter más general, hacia conceptos más específicos, como funciones, procesos o atributos particulares**. Este enfoque resulta especialmente relevante en disciplinas como la bioinformática, donde ontologías como _Gene Ontology_ emplean DAGs para modelar relaciones semánticas entre términos biológicos, permitiendo inferencias, comparaciones y cálculos de similitud basados en la estructura conceptual del dominio [12] [13] [14].

<figure style="text-align:center;">
    <img src="Images/GOGraph.png" width="30%">
    <figcaption>Figura 3: GO Graph [15]</figcaption>
</figure>

_Gene Ontology (GO)_ es una ontología biológica diseñada para la centralización, estandarización y organización del conocimiento biológico mediante un conjunto controlado de conceptos bien definidos y relaciones formales entre ellos. Su objetivo principal es proporcionar un lenguaje común que permita describir de manera consistente las funciones génicas a través de distintas especies y bases de datos. La ontología se divide en tres dominios principales, cada uno de los cuales representa un aspecto distinto de la biología molecular [12] [13]:

   1. **Biological Process (BP)**: Describe los procesos biológicos en los que participa un gen o producto génico, tales como la división celular, la señalización o el metabolismo.
   2. **Molecular Function (MF)**: Representa las actividades moleculares específicas que realiza un producto génico, por ejemplo, la unión a una molécula o la actividad catalítica.
   3. **Cellular Component (CC)**: Indica la localización celular donde actúa el producto génico, como el núcleo, la membrana o un complejo proteico.

Cada uno de estos dominios está organizado de forma independiente como un grafo dirigido acíclico (DAG), donde los conceptos (términos GO) se relacionan entre sí mediante relaciones semánticas jerárquicas, como **\_is_a\_** o**_part\_of_**. Esta estructura permite que un mismo término pueda tener múltiples padres, reflejando la complejidad inherente de los sistemas biológicos. Gracias a esta organización, _Gene Ontology_ facilita tareas como la anotación funcional, el análisis comparativo entre especies y el cálculo de similitud semántica entre genes o términos, siendo un componente fundamental en numerosas aplicaciones bioinformáticas [12] [13] [14].

## Métricas de similitud semántica

En la versión **0.3.0 de GO3**, se implementan diversas métricas para el cálculo de la **similitud semántica entre términos de *Gene Ontology (GO)***. Estas métricas permiten cuantificar cuán relacionados están dos conceptos biológicos dentro de la ontología y se apoyan principalmente en dos pilares teóricos [16] [17]:

- **Contenido de Información (Information Content, IC)**
- **Topología del grafo DAG de Gene Ontology**

### Contenido de Información (IC)

El **contenido de información** mide cuán específico o informativo es un término dentro de la ontología. Formalmente, se define como:


$IC(t) = -\log p(t)$

donde $p(t)$ corresponde a la probabilidad de ocurrencia del término $t$ en un conjunto de anotaciones biológicas (por ejemplo, archivos `.gaf`).  
Términos más generales presentan valores bajos de IC, mientras que términos más específicos presentan valores altos.

### Topología del DAG

La **topología** hace referencia a la **estructura jerárquica del grafo dirigido acíclico (DAG)** que define a Gene Ontology. En este contexto:

- Los nodos representan términos GO.
- Las aristas representan relaciones semánticas (por ejemplo, `is_a`, `part_of`).
- Un término puede tener múltiples padres.
- La similitud se calcula considerando rutas, ancestros comunes y pesos estructurales.

Las métricas topológicas no dependen directamente de la frecuencia de anotaciones, sino de la posición relativa de los términos en el grafo.

### Notación

Sean:
- $t_1, t_2$: Términos GO.
- $ A(t) $: Conjunto de ancestros de \( t \) (incluido el término).
- $ \text{MICA}(t_1, t_2) $: Ancestro común más informativo.
- $ IC(t) $: Contenido de información del término.


## Métricas *pairwise* (término a término) [17]

### Resnik

$\text{Sim}_{Resnik}(t_1, t_2) = IC(\text{MICA}(t_1, t_2))$

Métrica no normalizada basada exclusivamente en el IC del ancestro común más informativo.

### Lin

$\text{Sim}_{Lin}(t_1, t_2) = \frac{2 \cdot IC(\text{MICA})}{IC(t_1) + IC(t_2)}$

Métrica normalizada en el rango $[0,1]$.

### Jiang–Conrath (JC)

$\text{Dist}_{JC}(t_1, t_2) = IC(t_1) + IC(t_2) - 2 \cdot IC(\text{MICA})$

Transformación común a similitud:

$\text{Sim}_{JC} = \frac{1}{1 + \text{Dist}_{JC}}$

### SimRel

$
\text{Sim}_{SimRel}(t_1, t_2) =
\frac{2 \cdot IC(\text{MICA})}
{IC(t_1) + IC(t_2)}
\cdot \left(1 - e^{-IC(\text{MICA})}\right)
$

Refuerza la contribución de ancestros altamente informativos.

### Information Coefficient (ICCoef)

$
\text{Sim}_{ICCoef}(t_1, t_2) =
\frac{IC(\text{MICA})}{\max(IC(t_1), IC(t_2))}
$

### GraphIC

$
\text{Sim}_{GraphIC}(t_1, t_2) =
\frac{\sum_{a \in A(t_1) \cap A(t_2)} IC(a)}
{\sum_{a \in A(t_1) \cup A(t_2)} IC(a)}
$

Interpretado como un índice de Jaccard ponderado por IC.

### Wang

Sea $ S_t(a) $ la contribución semántica de un ancestro $ a $ al término $ t $:

$
\text{Sim}_{Wang}(t_1, t_2) =
\frac{\sum_{a \in A(t_1) \cap A(t_2)} [S_{t_1}(a) + S_{t_2}(a)]}
{\sum_{a \in A(t_1)} S_{t_1}(a) + \sum_{a \in A(t_2)} S_{t_2}(a)}
$

Métrica puramente topológica, independiente de anotaciones.

### TopoICSim

$
\text{Sim}_{TopoICSim}(t_1, t_2) =
\sum_{a \in A(t_1) \cap A(t_2)}
w(a) \cdot IC(a)
$

Combina información estructural y semántica.

## Métodos *groupwise* (conjuntos de términos) [17]

Sean:
- $ T_1, T_2 $: conjuntos de términos GO
- $ sim(t,s) $: similitud *pairwise*

### Best Match Average (BMA)

$
\text{Sim}_{BMA}(T_1, T_2) =
\frac{1}{2}
\left(
\frac{\sum_{t \in T_1} \max_{s \in T_2} sim(t,s)}{|T_1|}
+
\frac{\sum_{s \in T_2} \max_{t \in T_1} sim(t,s)}{|T_2|}
\right)
$

### Maximum

$
\text{Sim}_{Max}(T_1, T_2) =
\max_{t \in T_1, s \in T_2} sim(t,s)
$


### Average

$
\text{Sim}_{Avg}(T_1, T_2) =
\frac{1}{|T_1||T_2|}
\sum_{t \in T_1} \sum_{s \in T_2} sim(t,s)
$


### Hausdorff

$
\text{Sim}_{Hausdorff}(T_1, T_2) =
\min
\left(
\max_{t \in T_1} \min_{s \in T_2} sim(t,s),
\max_{s \in T_2} \min_{t \in T_1} sim(t,s)
\right)
$

### SimGIC

$
\text{Sim}_{SimGIC}(T_1, T_2) =
\frac{\sum_{a \in A(T_1) \cap A(T_2)} IC(a)}
{\sum_{a \in A(T_1) \cup A(T_2)} IC(a)}
$

Índice de cobertura semántica ponderado por IC.

## ¿Qué tipo de representaciones se utilizan para la **identificación de genes**? ¿Cuáles son compatibles y cómo afectan al resultado?

Una consideración relevante es la representación utilizada para identificar las entidades sobre las cuales se aplica el cálculo de cualquier métrica de similitud semántica. En este contexto, se distinguen principalmente dos tipos de entidades:
   1. **Términos**: Corresponden a los _Gene Ontology Terms_ definidos en el archivo _go-basic.obo_, los cuales se representan mediante la nomenclatura GO:\[ID\]. Este archivo fuente proporciona las relaciones semánticas _is\_a_ y _part\_of_, necesarias para la construcción del _GO DAG (Gene Ontology Directed Acyclic Graph)_, es decir, un grafo dirigido acíclico que organiza los conceptos desde términos más generales hasta otros más específicos.
   2. **Genes**: Los genes se definen formalmente como secciones del ADN que codifican productos funcionales, generalmente proteínas, las cuales participan en distintos procesos biológicos dentro de un organismo. Dado que existen múltiples organizaciones y bases de datos que gestionan identificadores génicos, **determinar un identificador único universal resulta complejo**. No obstante, uno de los esquemas más ampliamente utilizados es _Entrez Gene ID_, aunque no es el único disponible.

En el contexto de _Gene Ontology_, las anotaciones génicas representan la asociación entre un gen (o producto génico) y uno o más términos GO, indicando qué función realiza, en qué proceso biológico participa o en qué componente celular actúa dicho producto. Estas anotaciones constituyen la base empírica sobre la cual se calculan métricas de similitud semántica entre términos y genes. El formato estándar para representar estas asociaciones es el archivo _GAF (Gene Annotation Format)_, ampliamente utilizado por el _Gene Ontology Consortium_. Un archivo _GAF_ permite **vincular de manera estructurada identificadores génicos con términos _GO_**, incorporando además información sobre evidencia, procedencia y contexto biológico. 

Desde un punto de vista estructural, un archivo GAF corresponde a **un archivo de columnas separadas por tabulaciones**, donde cada fila representa una anotación individual. A continuación se describen los campos más relevantes del formato GAF 2.2, utilizados en este estudio.
   1. **DB**: Indica la base de datos de origen del identificador del gen o proteína (por ejemplo, UniProtKB, NCBI, Ensembl). Este campo permite contextualizar el sistema de identificación utilizado.
   2. **DB Object ID**: Identificador único del gen o producto génico dentro de la base de datos especificada. Es el campo principal para distinguir entidades biológicas de forma inequívoca.
   3. **DB Object Symbol**: Símbolo abreviado y legible del gen o proteína, comúnmente utilizado en literatura científica. No necesariamente es único, pero facilita la interpretación humana.
   4. **Qualifier**: Modificador que altera el significado de la anotación. Puede indicar, por ejemplo, negación (NOT) o relaciones especiales entre el gen y el término GO.
   5. **GO ID**: Identificador del término de Gene Ontology asociado a la anotación, siguiendo la nomenclatura estándar GO:\[ID\].
   6. **DB:Reference**: Referencias bibliográficas o identificadores de bases de datos que respaldan la anotación, tales como artículos científicos o registros curatoriales.
   7. **Evidence Code**: Código que describe el tipo de evidencia utilizada para asignar el término GO al gen (por ejemplo, evidencia experimental, inferida por similitud, computacional, etc.).
   8. **With / From**: Identificadores adicionales utilizados para inferir o respaldar la anotación, especialmente en casos donde la evidencia es indirecta.
   9. **Aspect** Indica el dominio de la ontología al que pertenece el término GO: BP: Biological Process MF: Molecular Function CC: Cellular Component
   10. **DB Object Name**: Nombre descriptivo del gen o proteína, generalmente más extenso que el símbolo.
   11. **DB Object Synonym** Lista de sinónimos asociados al objeto biológico, separados por un delimitador estándar.
   12. **DB Object Type**: Tipo de entidad anotada, como gene, protein, RNA o protein complex.
   13. **Taxon**: Identificador taxonómico del organismo al que pertenece el gen o producto génico, permitiendo distinguir anotaciones entre especies.
   14. **Date**: Fecha en la que la anotación fue creada o actualizada, expresada en formato estándar.
   15. **Assigned By**: Organización o base de datos responsable de la curaduría y asignación de la anotación.
   16. **Annotation Extension**: Permite agregar información contextual adicional a la anotación, refinando su significado mediante extensiones semánticas.
   17. **Gene Product Form ID**: Identifica una forma específica del producto génico (por ejemplo, isoformas proteicas).

En terminos prácticos esto **solo implica que buscamos determinar la simbología del objeto dentro de solamente el identificador asociado**, por ende, he decidido crear una función de apoyo que permita transformar cualquier identificador de genes al que corresponde según el archivo que se ha presentado.

## ¿La construcción de los _DAG_ es compatible únicamente con _go-basic.obo_ y archivos _.gaf_?

Sí. A partir de la revisión interna del código, se determinó que la construcción del _DAG de Gene Ontology_ es compatible exclusivamente con archivos en formato _.obo_ y _.gaf._ Esto se debe a que ambos formatos contienen información estructural y semántica esencial para el correcto funcionamiento de las funciones de cálculo de similitud implementadas en el paquete.

Dado que este punto es acotado, a continuación se presenta una descripción general del formato .obo, enfocada únicamente en los campos semánticos relevantes que permiten modelar la ontología y construir el grafo.

Un archivo .obo define los términos de Gene Ontology y las relaciones entre ellos. Conceptualmente, cada término puede describirse mediante los siguientes atributos:

1. **id**: Identificador único del término GO, utilizado como referencia principal dentro de la ontología.
2. **name**: Nombre descriptivo del término, que resume el concepto biológico que representa.
3. **namespace**: Dominio de la ontología al que pertenece el término: Biological Process (BP), Molecular Function (MF) o Cellular Component (CC).
4. **definition**: Definición formal del término, proporcionando contexto semántico preciso.
5. **parents**: Identificadores de los términos padres, que establecen relaciones jerárquicas hacia conceptos más generales.
6. **children**: Identificadores de los términos hijos, representando conceptos más específicos dentro de la ontología.
7. **depth (opcional)**: Profundidad del término dentro del DAG, medida desde un concepto raíz.
8. **level (opcional)**: Nivel jerárquico del término, utilizado como una medida alternativa de su posición estructural.
9. **is_obsolete**: Indica si el término ha sido marcado como obsoleto y ya no debe utilizarse en nuevas anotaciones.
10. **alt_ids**: Identificadores alternativos que el término pudo haber tenido en versiones anteriores de la ontología.
11. **replaced_by (opcional)**: Identificador del término que reemplaza a uno obsoleto, cuando aplica.
12. **consider**: Lista de términos relacionados que pueden considerarse como alternativas semánticas.
13. **synonyms**: Sinónimos asociados al término, útiles para ampliar la interpretación semántica.
14. **xrefs**: Referencias cruzadas a otras bases de datos u ontologías externas.
15. **relationships**: Relaciones semánticas entre términos, como is_a o part_of, que permiten construir el grafo dirigido acíclico.
16. **comment (opcional)**: Comentarios adicionales que aportan contexto o aclaraciones curatoriales.


## ¿Es compatible con el estudio de similitud en otras especies?
Si, dado que los _.gaf_ son archivos asociados a cada especie. Se provee código de ejemplo.

In [1]:
import go3
from itertools import combinations
from statistics import mean

def print_pairs(pairs, columns=3):
    print(f"Total pairs: {len(pairs)}")
    for i in range(0, len(pairs), columns):
        chunk = pairs[i:i+columns]
        line = " | ".join(f"{a}-{b}" for a, b in chunk)
        print(line)

def print_pair_scores(pairs, scores, precision=4):
    print(f"Scores (n={len(scores)})")
    for (a, b), s in zip(pairs, scores):
        print(f"{a:>8} vs {b:<8} : {s:.{precision}f}")
    if scores:
        print(f"min={min(scores):.{precision}f}  max={max(scores):.{precision}f}  mean={mean(scores):.{precision}f}")

# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go.obo")
annotations = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\tair.gaf")

# Build IC Counter
counter = go3.build_term_counter(annotations)

genes = ["ELI3-1","CAM2","CCR1","TSA1","ECS1","HOG1","AT-P4H-1","HLECRK","LTI30"]
pairs = list(combinations(genes, 2))
print_pairs(pairs)

scores = go3.compare_gene_pairs_batch(pairs, "BP", "wang", "bma", counter)
print_pair_scores(pairs, scores)

sim = go3.compare_genes('MDN1','LTI30', "BP", "wang", "bma", counter)
print(f"MDN1 vs LTI30 (BP/wang/bma): {sim:.4f}")


Total pairs: 36
ELI3-1-CAM2 | ELI3-1-CCR1 | ELI3-1-TSA1
ELI3-1-ECS1 | ELI3-1-HOG1 | ELI3-1-AT-P4H-1
ELI3-1-HLECRK | ELI3-1-LTI30 | CAM2-CCR1
CAM2-TSA1 | CAM2-ECS1 | CAM2-HOG1
CAM2-AT-P4H-1 | CAM2-HLECRK | CAM2-LTI30
CCR1-TSA1 | CCR1-ECS1 | CCR1-HOG1
CCR1-AT-P4H-1 | CCR1-HLECRK | CCR1-LTI30
TSA1-ECS1 | TSA1-HOG1 | TSA1-AT-P4H-1
TSA1-HLECRK | TSA1-LTI30 | ECS1-HOG1
ECS1-AT-P4H-1 | ECS1-HLECRK | ECS1-LTI30
HOG1-AT-P4H-1 | HOG1-HLECRK | HOG1-LTI30
AT-P4H-1-HLECRK | AT-P4H-1-LTI30 | HLECRK-LTI30
Scores (n=36)
  ELI3-1 vs CAM2     : 0.1154
  ELI3-1 vs CCR1     : 0.3834
  ELI3-1 vs TSA1     : 0.3042
  ELI3-1 vs ECS1     : 0.3620
  ELI3-1 vs HOG1     : 0.0290
  ELI3-1 vs AT-P4H-1 : 0.1324
  ELI3-1 vs HLECRK   : 0.0000
  ELI3-1 vs LTI30    : 0.2369
    CAM2 vs CCR1     : 0.1158
    CAM2 vs TSA1     : 0.0731
    CAM2 vs ECS1     : 0.1228
    CAM2 vs HOG1     : 0.0592
    CAM2 vs AT-P4H-1 : 0.1410
    CAM2 vs HLECRK   : 0.0000
    CAM2 vs LTI30    : 0.0611
    CCR1 vs TSA1     : 0.1732
    CCR1 v

## En R existen otros paquetes: ¿Qué diferencias de velocidad y resultados ofrecen? ¿Son comparables entre sí? - ¿Por qué no se realizaron más pruebas comparativas con implementaciones en otros lenguajes?

La comparación entre **GO3** y alternativas en **R** se encuentra actualmente en desarrollo. Sin embargo, una comparación “uno a uno” puede presentar complicaciones metodológicas, ya que **las diferencias observadas en los resultados no necesariamente reflejan errores**, sino que pueden deberse a que cada implementación utiliza **fuentes de información y decisiones de preprocesamiento distintas**.

En particular, es esperable que existan diferencias cuando cambian uno o más de los siguientes componentes:

- **Versiones y variantes de la ontología GO** (por ejemplo, cambios en `go-basic.obo`, términos obsoletos, *alt_id*, relaciones consideradas).
- **Fuentes de anotación** y sus criterios de filtrado (por ejemplo, archivos `.gaf`, evidencias incluidas/excluidas, anotaciones negadas con `NOT`).
- **Cálculo del contenido de información (IC)**, que depende de cómo se obtienen probabilidades/frecuencias a partir de las anotaciones (y de si se realiza propagación hacia ancestros).
- **Convenciones internas del grafo** (qué relaciones se incorporan, cómo se define el conjunto de ancestros, tratamiento de múltiples padres).
- **Estrategias *groupwise*** (por ejemplo, BMA vs promedio global), que pueden producir diferencias incluso usando la misma métrica *pairwise*.

### Sobre la ausencia de más pruebas comparativas entre lenguajes

No se realizaron más comparaciones entre lenguajes por dos razones principales:

1. **Equidad experimental y reproducibilidad**: para que una comparación sea justa, se requiere igualar cuidadosamente (i) ontología, (ii) anotaciones, (iii) filtrado, (iv) cálculo de IC, y (v) método *groupwise*. Si estos elementos no se controlan, una diferencia numérica puede atribuirse erróneamente al lenguaje o a la implementación.

2. **Diferencias de interfaz y dependencias**: en la práctica, distintos paquetes exponen configuraciones, formatos y supuestos no equivalentes (por ejemplo, cómo se manejan identificadores, especies, fuentes de anotación o relaciones del DAG), lo que incrementa el costo de construir un *benchmark* comparable y verificable.

En consecuencia, el foco del análisis comparativo se orienta a construir primero un escenario controlado donde las fuentes y supuestos sean lo más equivalentes posible, y recién entonces evaluar diferencias de **resultado** y **rendimiento**.


In [ ]:
# Libraries.
import go3
from itertools import combinations
from statistics import mean

def print_scores(pairs, scores, precision=4, max_rows=50):
    print(f"Pairs: {len(pairs)}")
    for i, ((a, b), s) in enumerate(zip(pairs, scores)):
        if i >= max_rows:
            remaining = len(pairs) - max_rows
            if remaining > 0:
                print(f"... {remaining} more")
            break
        print(f"{a} vs {b} : {s:.{precision}f}")
    if scores:
        print(f"min={min(scores):.{precision}f}  max={max(scores):.{precision}f}  mean={mean(scores):.{precision}f}")

# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go-basic.obo")
annotations_h = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\goa_human.gaf")

# Build IC Counter
counter = go3.build_term_counter(annotations_h)

terms = ["GO:0098596","GO:0098720","GO:1990000","GO:0007497","GO:0003429","GO:2000348","GO:0009201","GO:0045774","GO:0014877","GO:0030241"]
pairs = list(combinations(terms, 2))

scores = go3.batch_similarity([a for a, _ in pairs], [b for _, b in pairs], "wang", counter)
print_scores(pairs, scores, precision=3)

sim = go3.semantic_similarity("GO:0098596", "GO:0098720", 'wang', counter)
print(f"GO:0098596 vs GO:0098720 (wang): {sim:.4f}")


Pairs: 45
GO:0098596 vs GO:0098720 : 0.0237
GO:0098596 vs GO:1990000 : 0.0304
GO:0098596 vs GO:0007497 : 0.0440
GO:0098596 vs GO:0003429 : 0.0049
GO:0098596 vs GO:2000348 : 0.0328
GO:0098596 vs GO:0009201 : 0.0258
GO:0098596 vs GO:0045774 : 0.0224
GO:0098596 vs GO:0014877 : 0.0231
GO:0098596 vs GO:0030241 : 0.0055
GO:0098720 vs GO:1990000 : 0.0855
GO:0098720 vs GO:0007497 : 0.0109
GO:0098720 vs GO:0003429 : 0.0204
GO:0098720 vs GO:2000348 : 0.0351
GO:0098720 vs GO:0009201 : 0.0426
GO:0098720 vs GO:0045774 : 0.0213
GO:0098720 vs GO:0014877 : 0.0161
GO:0098720 vs GO:0030241 : 0.0112
GO:1990000 vs GO:0007497 : 0.0145
GO:1990000 vs GO:0003429 : 0.0241
GO:1990000 vs GO:2000348 : 0.0557
GO:1990000 vs GO:0009201 : 0.1328
GO:1990000 vs GO:0045774 : 0.0254
GO:1990000 vs GO:0014877 : 0.0202
GO:1990000 vs GO:0030241 : 0.0951
GO:0007497 vs GO:0003429 : 0.0218
GO:0007497 vs GO:2000348 : 0.0158
GO:0007497 vs GO:0009201 : 0.0120
GO:0007497 vs GO:0045774 : 0.0103
GO:0007497 vs GO:0014877 : 0.0147
GO:0

In [ ]:
# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go-basic.obo")
annotations_h = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\goa_human.gaf")

# Build IC Counter
counter = go3.build_term_counter(annotations_h)

terms = ["VSTM2L","N4BP1","GIT2","OMA1","AP2A1","SSU72","KLHL18","FOXH1","MRPS15","TTC19"]
pairs = list(combinations(terms, 2))

scores = go3.compare_gene_pairs_batch(pairs, "BP", "wang", "bma", counter)
print_scores(pairs, scores, precision=3)

Pairs: 45
VSTM2L vs N4BP1 : 0.1197
VSTM2L vs GIT2 : 0.1441
VSTM2L vs OMA1 : 0.1015
VSTM2L vs AP2A1 : 0.0704
VSTM2L vs SSU72 : 0.0561
VSTM2L vs KLHL18 : 0.0647
VSTM2L vs FOXH1 : 0.0645
VSTM2L vs MRPS15 : 0.0247
VSTM2L vs TTC19 : 0.0349
N4BP1 vs GIT2 : 0.1413
N4BP1 vs OMA1 : 0.1362
N4BP1 vs AP2A1 : 0.1552
N4BP1 vs SSU72 : 0.1226
N4BP1 vs KLHL18 : 0.0911
N4BP1 vs FOXH1 : 0.1458
N4BP1 vs MRPS15 : 0.0305
N4BP1 vs TTC19 : 0.0750
GIT2 vs OMA1 : 0.1404
GIT2 vs AP2A1 : 0.2834
GIT2 vs SSU72 : 0.0799
GIT2 vs KLHL18 : 0.1078
GIT2 vs FOXH1 : 0.1583
GIT2 vs MRPS15 : 0.0459
GIT2 vs TTC19 : 0.0872
OMA1 vs AP2A1 : 0.1549
OMA1 vs SSU72 : 0.1698
OMA1 vs KLHL18 : 0.2484
OMA1 vs FOXH1 : 0.1239
OMA1 vs MRPS15 : 0.2143
OMA1 vs TTC19 : 0.2164
AP2A1 vs SSU72 : 0.0905
AP2A1 vs KLHL18 : 0.1654
AP2A1 vs FOXH1 : 0.1098
AP2A1 vs MRPS15 : 0.0541
AP2A1 vs TTC19 : 0.1494
SSU72 vs KLHL18 : 0.1279
SSU72 vs FOXH1 : 0.2375
SSU72 vs MRPS15 : 0.2755
SSU72 vs TTC19 : 0.0590
KLHL18 vs FOXH1 : 0.0675
KLHL18 vs MRPS15 : 0.2182


In [ ]:
# Load Gene Ontology (GO) terms and annotations
go_terms = go3.load_go_terms(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\go-basic.obo")
annotations_h = go3.load_gaf(r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\tair.gaf")

# Build IC Counter
counter = go3.build_term_counter(annotations_h)

terms = ["ELI3-1","CAM2","CCR1","TSA1","ECS1","HOG1","AT-P4H-1","HLECRK","LTI30"]
pairs = list(combinations(terms, 2))

scores = go3.compare_gene_pairs_batch(pairs, "BP", "wang", "bma", counter)
print_scores(pairs, scores, precision=3)

Pairs: 36
ELI3-1 vs CAM2 : 0.1154
ELI3-1 vs CCR1 : 0.3834
ELI3-1 vs TSA1 : 0.3164
ELI3-1 vs ECS1 : 0.3620
ELI3-1 vs HOG1 : 0.0290
ELI3-1 vs AT-P4H-1 : 0.1324
ELI3-1 vs HLECRK : 0.0000
ELI3-1 vs LTI30 : 0.2369
CAM2 vs CCR1 : 0.1158
CAM2 vs TSA1 : 0.0784
CAM2 vs ECS1 : 0.1228
CAM2 vs HOG1 : 0.0592
CAM2 vs AT-P4H-1 : 0.1410
CAM2 vs HLECRK : 0.0000
CAM2 vs LTI30 : 0.0611
CCR1 vs TSA1 : 0.1776
CCR1 vs ECS1 : 0.2696
CCR1 vs HOG1 : 0.0283
CCR1 vs AT-P4H-1 : 0.0477
CCR1 vs HLECRK : 0.0000
CCR1 vs LTI30 : 0.4755
TSA1 vs ECS1 : 0.3457
TSA1 vs HOG1 : 0.0619
TSA1 vs AT-P4H-1 : 0.1407
TSA1 vs HLECRK : 0.0000
TSA1 vs LTI30 : 0.3522
ECS1 vs HOG1 : 0.0557
ECS1 vs AT-P4H-1 : 0.0206
ECS1 vs HLECRK : 0.0000
ECS1 vs LTI30 : 0.3919
HOG1 vs AT-P4H-1 : 0.0190
HOG1 vs HLECRK : 0.0000
HOG1 vs LTI30 : 0.0197
AT-P4H-1 vs HLECRK : 0.0000
AT-P4H-1 vs LTI30 : 0.0201
HLECRK vs LTI30 : 0.0000
min=0.0000  max=0.4755  mean=0.1272


## ¿Existe alguna forma de administrar o normalizar los identificadores de genes?
De forma nativa en la librería no existe una función dedicada a que desde un identificador cualquiera haga un mapeado a los identificadores asociados a los gaf, no obstante, eso no implica que sea imposible. Desde mi parte decidí proporcionar un código que pueda realizar esta tarea con cierto grado de exito.

In [13]:
%%time
%pip install mygene
%pip install gprofiler-official

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
CPU times: total: 46.9 ms
Wall time: 7.74 s



[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
%%time
import pandas as pd
from mygene import MyGeneInfo
import re

def mapeo_genes(
    genes_ids,
    gaf_path,
    scopes_default=("uniprot", "entrezgene", "ensembl.gene", "symbol", "name"),
    fields="symbol"
):
    """
    Normaliza una lista de identificadores génicos hacia símbolos canónicos,
    usando un archivo GAF como referencia primaria.
    """

    # ─────────────────────────────
    # 1. Leer GAF
    # ─────────────────────────────
    gaf = pd.read_csv(
        gaf_path,
        sep="\t",
        comment="!",
        header=None,
        dtype=str
    )

    gaf.columns = [
        "DB", "DB_ID", "Symbol", "Qualifier", "GO_ID",
        "Reference", "Evidence", "With_From", "Aspect",
        "Name", "Synonyms", "Type", "Taxon", "Date", "Assigned_By", 
        "Annotation_Extension", "Gene_Product_Form_ID"
    ]

    # ─────────────────────────────
    # 2. Conjuntos de referencia
    # ─────────────────────────────
    valid_symbols = set(gaf["Symbol"].dropna().unique())
    db_to_scope = {
        "UniProtKB": "uniprot",
        "NCBI_GeneID": "entrezgene",
        "Ensembl": "ensembl.gene"
    }

    # detectar especie desde taxon
    taxon = (
        gaf["Taxon"]
        .dropna()
        .iloc[0]
        .replace("taxon:", "")
    )

    mg = MyGeneInfo()

    # ─────────────────────────────
    # 3. Separar IDs válidos / inválidos
    # ─────────────────────────────
    genes_validos = []
    genes_a_mapear = []

    for g in genes_ids:
        if g in valid_symbols:
            genes_validos.append(g)
        else:
            genes_a_mapear.append(g)

    # ─────────────────────────────
    # 4. Mapear solo los no válidos
    # ─────────────────────────────
    mapping = {}

    if genes_a_mapear:
        res = mg.querymany(
            genes_a_mapear,
            scopes=scopes_default,
            fields=fields,
            species=taxon
        )

        for r in res:
            if not r.get("notfound", False):
                mapping[r["query"]] = r.get("symbol")

    # ─────────────────────────────
    # 5. Construir salida final
    # ─────────────────────────────
    final_mapping = {}

    for g in genes_ids:
        if g in genes_validos:
            final_mapping[g] = g
        else:
            final_mapping[g] = mapping.get(g)

    lista_final = list(
        {v for v in final_mapping.values() if v is not None}
    )

    return {
        "mapping": final_mapping,
        "final_gene_list": lista_final,
        "species_taxon": taxon
    }

# Ejemplo humano y Thaliana.
ids_human = ["VSTM2L","N4BP1","GIT2","FAM160A2","OMA1","TBCK","CCDC47","STAG2","CCPG1",
             "ETAA1","HOOK1","PDK4","LMBRD1","MAGEF1","C1QTNF1","PPP2R2D","PIK3AP1",
             "GLIPR1","TMBIM4","MS4A1","FBLIM1","ARID1A","ACVRL1","MAN1C1","TMEM129",
             "KRT23","RDH11","NUP54","ZBTB7A","LIN7C","EGFL7","ABHD4","AMDHD2","RNF146",
             "DOCK7","STARD4","RCOR3","CAMTA1","COX18","SP1","AKT1S1","LONP2","THNSL2",
             "XPO4","TSPAN15","FBXO30","ANGPTL4","ARHGAP24","SSH1","ECHDC2","TMX3","GNA13",
             "SVIP","ZC3H12A","UQCR10","HEATR5A","CCM2","DYNLRB1","TM7SF3","ATAD1","FAM53C",
             "TMEM167A","CMTM6","RHOBTB3","IRS2","SLC44A2","NPDC1","ASAP1","NUS1","SCYL1",
             "DHFR","KPNA4","UFSP2","AIG1","SPCS3","APPL2","TMEM50A","TRAPPC5","ING4",
             "GINS2","ITM2B","NDUFA4","OST4","RPS27L","NAA20","UBL5","DNAJC10","SF3B5",
             "ALKBH7","MYOZ1","EIF2A","ERRFI1","PACS1","PCYOX1","HVCN1","STYXL1","GATAD2A",
             "BRWD1","GPM6B","KLF11","EIF4ENIF1","USP39","NSFL1C","ARRDC1","PFDN2","STARD3NL",
             "RANGRF","ECHDC1","PPTC7","TMED4","WIPF2","ARHGAP17","CAPRIN2","ENAH","RLIM",
             "RAB25","DDR2","BAG3","TPCN2","WBP11","ESAM","ANAPC16","PBRM1","SLC44A1","ERGIC1",
             "GOLPH3","HAUS1","NDUFB10","MYLIP","HSD17B12","GNAQ","ECSIT","DENR","PPHLN1",
             "UBQLN1","PYCR2","NAA50","DDOST","IER3IP1","MRPL55","NEXN","PAPPA","PRKCE","ABHD12",
             "DTX3","AK3","UBE2F","B3GALT6","PPA2","PIH1D1","PIM3","RAB10","NSMCE1","UFM1",
             "PPME1","TIMP2","VPS29","HMG20A","CRK","ACACA","RIC8A","DDAH1","PIGT","ADAMTS2",
             "ZNF358","RASGRP4","SOCS3","MRPS22","GET4","B3GAT2","SMAP1","MRPS15","MACROD1",
             "SEC61A2","VWA1","HM13","FOXK1","MAFK","AP2A1","FUZ","BAZ1A","ARID2","PCGF3",
             "RRN3P2","KIF5A","ORMDL2","SLTM","NLRC3","NR2F6","CBLC","TAF1D","SLC2A4RG",
             "THOC7","RREB1","EPB41","UBR5","EXOSC3","IRF2BP2","KLHL22","ZNF224","CDC42SE1",
             "USP13","LY75","WWC3","UBR4","FECH","RPS6KA6","TBX5","COL17A1","HK2","SPRED1",
             "LYST","PRRC1","AMMECR1","ZNF117","GLIS3","DMRT3","ENOPH1","BEND3","MRPS26",
             "NDUFC2","ZNF497","PPP2R2C","RNF34","RP9","GON4L","MSTO1","SEMA6D","USP42",
             "PPP1R3F","AGAP3","ALG5","MYBBP1A","HES6","MTHFD1L","PGLS","PTPMT1","FOXO3",
             "NR2C2","MALAT1","ZNF672","MMP24","VAMP8","ZRANB1","ISYNA1","RNF169","HCN3",
             "ZBTB46","DAGLB","LGALS8","ARID1B","SPAG9","CERCAM","EFNA5","TAF6L","TRIM8",
             "TMEM165","UBXN6","MTFP1","SGTB","PET117","SSU72","TLE3","CAMK2N2","ECE2","DOCK6",
             "RWDD2B","SLC25A37","CRLS1","IREB2","AKAP12","PIK3R5","GPR12","PHF8","STX11","DLC1",
             "USP53","HELZ","DNAH14","PTBP2","KIF13A","TRPC5","KRT82","SNX25","PSMD13","HLCS","AGGF1",
             "BRWD3","CD28","PTCHD1","UQCRB","DYNC1H1","CLN6","MED26","PTPRS","SNX14","ZDHHC22","MTPAP",
             "ANAPC7","MAT2B","WDR4","B3GNT9","ZBTB26","NXF3","TUBE1","KCTD5","DDX54","TRIM69","PPCS",
             "KANK4","C1orf56","GADD45GIP1","TIMM22","SLC33A1","FOXH1","SAMD1","MLXIPL","MRPL35","EGLN2",
             "PARP8","PHF20","NCS1","E2F6","NBR1","ALG13","ERI1","MYO9A","SLC6A11","GNAT1","KLHL18","COL13A1",
             "HIGD1C","FAM9B","CAND1","CAST","SMAD9","MDM4","RBM12","ZNF626","RGMB","GPATCH2","NAP1L4",
             "LSM2","DLG1","INPP5F","NRXN1","RELL1","ZFPM2","SNRPN","AKAP5","ZNF148","CDK19","VGLL3",
             "USP8","TACC2","MRPS10","NR2C2AP","FA2H","DAZAP1","CCDC136","SAP18","DCLRE1C","RPS23","IP6K2",
             "SCML1","SHARPIN","TM9SF1","TOX4","GPRC5B","MRPL10","NMRAL1","PLIN3","TP53I13","TES","PSAT1",
             "BCL2L10","HECTD2","SERP1","CRIPT","THAP3","RPS3A","SMAD3","MPRIP","NUP107","SPRYD3","SNX12",
             "PANX1","RPRM","ABCA7","MDFIC","SAE1","CLCN2","TTC17","TTC19","SPESP1","MTRF1L","GALNT1",
             "FBXO8","NUDCD2","VIL1","UGCG","TMEM119","IL1RAP","RAPGEF1","RNF126","PPP2R3B","ABCG8",
             "TXNDC16","ZNF704","RAD50","CORO7","OSR2","TEKT5","RFX4","LGALS16","MED13L","THOC3","PSMA8",
             "TBX4","ZNF655","TMEM132B","KDM5D","KCNJ16","INSC","RABGAP1","HSBP1L1","MKRN1","CALM1","MCPH1",
             "SLC39A13","NEFL","KDM2B","MAGED2","PAPOLA","DDX20","THAP7","LGMN","DAB2IP","ACTG1","SNX9",
             "DNAJC4","TTC39C","FPR3","NADSYN1","TYW1","UBE2D4","ZCCHC3","ENSA","VTA1","PSMD1","BRD3",
             "TRDN","RSL1D1","ERMN","TERF1","B3GALNT2","BMPR2","TMSB10","FLYWCH2","YME1L1","SURF4","HNRNPL",
             "PLSCR4","TBRG4","PLEKHB2","STXBP5L","PCDHB15","KCNT2","WDR90","B4GALNT1","DBT","LNPEP","ZIC4",
             "MMAA","CELF3","NUP155","ARID5B","COQ2","AGPAT3","KCNH8","RPL4","KYNU","GNAL","DAP3","NRBF2","CARD17",
             "COL4A2","BEND5","BMP2K","NEK10","STAB2","NDUFV3","RAB28","MATR3","XPNPEP1","MAPK9","VNN1",
             "CETN1","INTS6","SETD4"]

ids_thaliana = ["AT4G16610","AT4G29010","AT2G34420","AT2G43080","AT3G12120","AT3G04720","AB000623","AT1G55020",
                "T04323","AT1G76100","AT3G15730","AT3G12500","AT5G38430","AT1G33960","AT1G32060","AT5G65110",
                "AT1G28670","AT4G39260","AT2G24180","AT2G26740","AT5G35630","AT3G15350","AT2G35790","AT3G06860",
                "Y11607","AT2G34660","AT2G33150","AT2G23430","AT2G39780","AT4G25100","AB003040","AT1G78380","AT1G37130",
                "Z56278","AT1G07890","X66017","AT4G39330","AT5G13930","AT3G20230","AT4G39980","AT1G31580","AT4G37980",
                "AT2G34770","AJ000470","L15389","AF062901","AT4G00860","AT3G57260","AT4G20360","AT3G16480","AT3G16520",
                "AT2G42010","AT2G02990","AT1G76490","X99793","AT4G38530","AT1G67980","AT5G39190","AT3G02520","AT3G59700",
                "AT4G09320","AT3G44300","AT5G24090","AT3G25230","AT1G08830","AT1G51680","AT1G05010","AT4G16760","AT5G42650",
                "AT5G05730","AT1G25220","AT3G29200","AT4G31500","AT3G13920","AT3G11170","Z26426","AT2G29450","AF087932",
                "AT5G02500","AT1G51760","AT3G45140","AT1G76680","AT2G37040","AT3G53260","AT1G56190","Y14590","AT3G30775",
                "AT5G42180","AT4G13940","AT3G54640","AT5G54810","AT3G50970","AT3G18780","AT1G16060","AT5G54160","X16432",
                "AT1G64280","AT5G66570","AT4G01850","AT1G01470","AT2G20340","AT3G47290","AT2G14610","AT1G75040","AT5G20620",
                "AT2G26330","AT2G25080","AT4G14980","AT1G67750","AT1G67550","AT5G26000","AT5G25980","AT2G05510","AT1G04820",
                "AT1G01480","AT3G28930","AT4G20890","AT2G45820","AT3G15210","AT5G47770","AT2G28740","AT2G14850","AT4G08500",
                "AT3G45640","AT5G52350","AT3G20600","AT5G06940","AT1G53840","AT2G41110","AT5G37770","AT2G41100","AT5G57560",
                "AT1G72260"]

res = mapeo_genes(ids_human, gaf_path=r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\goa_human.gaf")

print(res["mapping"])
print(res["final_gene_list"])
print(len(res["final_gene_list"]))
print(res["species_taxon"])

res = mapeo_genes(ids_thaliana, gaf_path=r"C:\Users\benja\Desktop\workspace\GO3_Study\Files\tair.gaf", scopes_default=["tair", "symbol", "ensembl.gene", "uniprot"])
print(res["mapping"])
print(res["final_gene_list"])
print(len(res["final_gene_list"]))
print(res["species_taxon"])


Input sequence provided is already in string format. No operation performed
1 input query terms found dup hits:	[('MALAT1', 8)]
2 input query terms found no hit:	['FAM160A2', 'CARD17']


{'VSTM2L': 'VSTM2L', 'N4BP1': 'N4BP1', 'GIT2': 'GIT2', 'FAM160A2': None, 'OMA1': 'OMA1', 'TBCK': 'TBCK', 'CCDC47': 'CCDC47', 'STAG2': 'STAG2', 'CCPG1': 'CCPG1', 'ETAA1': 'ETAA1', 'HOOK1': 'HOOK1', 'PDK4': 'PDK4', 'LMBRD1': 'LMBRD1', 'MAGEF1': 'MAGEF1', 'C1QTNF1': 'C1QTNF1', 'PPP2R2D': 'PPP2R2D', 'PIK3AP1': 'PIK3AP1', 'GLIPR1': 'GLIPR1', 'TMBIM4': 'TMBIM4', 'MS4A1': 'MS4A1', 'FBLIM1': 'FBLIM1', 'ARID1A': 'ARID1A', 'ACVRL1': 'ACVRL1', 'MAN1C1': 'MAN1C1', 'TMEM129': 'TMEM129', 'KRT23': 'KRT23', 'RDH11': 'RDH11', 'NUP54': 'NUP54', 'ZBTB7A': 'ZBTB7A', 'LIN7C': 'LIN7C', 'EGFL7': 'EGFL7', 'ABHD4': 'ABHD4', 'AMDHD2': 'AMDHD2', 'RNF146': 'RNF146', 'DOCK7': 'DOCK7', 'STARD4': 'STARD4', 'RCOR3': 'RCOR3', 'CAMTA1': 'CAMTA1', 'COX18': 'COX18', 'SP1': 'SP1', 'AKT1S1': 'AKT1S1', 'LONP2': 'LONP2', 'THNSL2': 'THNSL2', 'XPO4': 'XPO4', 'TSPAN15': 'TSPAN15', 'FBXO30': 'FBXO30', 'ANGPTL4': 'ANGPTL4', 'ARHGAP24': 'ARHGAP24', 'SSH1': 'SSH1', 'ECHDC2': 'ECHDC2', 'TMX3': 'TMX3', 'GNA13': 'GNA13', 'SVIP': 'SVIP

Input sequence provided is already in string format. No operation performed
14 input query terms found no hit:	['AB000623', 'T04323', 'Y11607', 'AB003040', 'Z56278', 'X66017', 'AJ000470', 'L15389', 'AF062901', '


{'AT4G16610': 'AT4G16610', 'AT4G29010': 'AIM1', 'AT2G34420': 'LHB1B2', 'AT2G43080': 'AT-P4H-1', 'AT3G12120': 'FAD2', 'AT3G04720': 'PR4', 'AB000623': None, 'AT1G55020': 'LOX1', 'T04323': None, 'AT1G76100': 'PETE1', 'AT3G15730': 'PLDALPHA1', 'AT3G12500': 'HCHIB', 'AT5G38430': 'RBCS1B', 'AT1G33960': 'AIG1', 'AT1G32060': 'PRK', 'AT5G65110': 'ACX2', 'AT1G28670': 'ARAB-1', 'AT4G39260': 'CCR1', 'AT2G24180': 'CYP71B6', 'AT2G26740': 'SEH', 'AT5G35630': 'GS2', 'AT3G15350': 'AT3G15350', 'AT2G35790': 'AT2G35790', 'AT3G06860': 'MFP2', 'Y11607': None, 'AT2G34660': 'ABCC2', 'AT2G33150': 'PKT3', 'AT2G23430': 'ICK1', 'AT2G39780': 'RNS2', 'AT4G25100': 'FSD1', 'AB003040': None, 'AT1G78380': 'GSTU19', 'AT1G37130': 'NIA2', 'Z56278': None, 'AT1G07890': 'APX1', 'X66017': None, 'AT4G39330': 'CAD9', 'AT5G13930': 'TT4', 'AT3G20230': 'AT3G20230', 'AT4G39980': 'DHS1', 'AT1G31580': 'ECS1', 'AT4G37980': 'ELI3-1', 'AT2G34770': 'FAH1', 'AJ000470': None, 'L15389': None, 'AF062901': None, 'AT4G00860': 'ATOZI1', 'AT3G57

## Referencias

  [1] Welch L, Lewitter F, Schwartz R, Brooksbank C, Radivojac P, Gaeta B, Schneider MV. Bioinformatics curriculum guidelines: toward a definition of core competencies. PLoS Comput Biol. 2014 Mar 6;10(3):e1003496. doi: 10.1371/journal.pcbi.1003496. PMID: 24603430; PMCID: PMC3945096.
  
  [2] Lesk AM (26 July 2013). "Bioinformatics". Encyclopaedia Britannica. Archived from the original on 14 April 2021. Retrieved 17 April 2017.
  
  [3] Xiong, J. (2006). Essential bioinformatics. En Cambridge University Press eBooks. https://doi.org/10.1017/cbo9780511806087
  
  [4] Alberts, B., Johnson, A., Lewis, J., Raff, M., Roberts, K., & Walter, P. (2007). Molecular Biology of the Cell. En W.W. Norton & Company eBooks. https://doi.org/10.1201/9780203833445

  [5] Malavé, Dr Antonio Alcalá (4 de noviembre de 2015). Genética de la emoción: El origen de la enfermedad. Penguin Random House Grupo Editorial España. ISBN 9788490692066. Consultado el 1 de octubre de 2019.

  [6] Diccionario de genética - ADN. Instituto Nacional del Cáncer (Estados Unidos). 20 de julio de 2012. Consultado el 21 de octubre de 2023.

  [7] Kennedy C, ter Huurne J, Berkhout M, Gruis N, Bastiaens M, Bergman W, Willemze R, Bavinck JN. Melanocortin 1 receptor (MC1R) gene variants are associated with an increased risk for cutaneous melanoma which is largely independent of skin type and hair color. J Invest Dermatol. 2001 Aug;117(2):294-300. doi: 10.1046/j.0022-202x.2001.01421.x. PMID: 11511307.

  [8] Khan Academy. (2025). Introducción a la expresión génica (dogma central). Khan Academy. https://es.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/intro-to-gene-expression-central-dogma Consultado el 10 de febrero de 2026.

  [9] crosspixel. (s. f.). Red haired girl with freckles on her face. Vector illustration. 123RF. https://es.123rf.com/photo_223752874_red-haired-girl-with-freckles-on-her-face-vector-illustration.html 

  [10] Cruse, D. A. (2011). Meaning in language: An introduction to semantics and pragmatics (3rd ed.). Oxford University Press.

  [11] Heim, I., & Kratzer, A. (1998). Semantics in generative grammar. Blackwell Publishing.

  [12] Ashburner M, Ball CA, Blake JA, Botstein D, Butler H, Cherry JM, Davis AP, Dolinski K, Dwight SS, Eppig JT, Harris MA, Hill DP, Issel-Tarver L, Kasarskis A, Lewis S, Matese JC, Richardson JE, Ringwald M, Rubin GM, Sherlock G. Gene ontology: tool for the unification of biology. The Gene Ontology Consortium. Nat Genet. 2000 May;25(1):25-9. doi: 10.1038/75556. PMID: 10802651; PMCID: PMC3037419.

  [13] Gene Ontology Resource. (s. f.). Gene Ontology Resource. https://www.geneontology.org/

  [14] Gene Ontology Consortium. (2023). The Gene Ontology resource: enriching a GOld mine. Nucleic Acids Research, 51(D1), D325–D334. https://doi.org/10.1093/nar/gkac1000

  [15] Baldock, R., & Burger, A. (2005). Anatomical ontologies: names and places in biology. Genome Biology, 6(4), 108. https://doi.org/10.1186/gb-2005-6-4-108

  [16] go3: A Fast and Lightweight Library for Semantic Similarity of GO Terms and Genes Jose L. Mellina-Andreu, Alejandro Cisterna-Garcia, Juan A. Botia bioRxiv 2025.09.04.669468; doi: https://doi.org/10.1101/2025.09.04.669468

  [17] GO3 documentation. (2025). https://go3.readthedocs.io/en/latest/